For our `ogbn-papers100M` experiments, we have 256GB memory. If you have enough memory, you can simply run `run_ogbn.py` for `ogbn-papers100M`. However, due to the hugh size of the dataset, we have to preprocess some data. It is recommended to run this jupyter notebook first before you run `run_100m.py`.

In [ ]:
import sys
sys.path.append('/home/chendi/ibmb')

In [ ]:
import numpy as np
import torch
import pickle

## load original data

In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset

In [ ]:
dataset = PygNodePropPredDataset(name="ogbn-papers100M", root='/nfs/students/qian')  # use your /path/to/data

splits = dataset.get_idx_split()
train_indices = splits['train'].numpy()
val_indices = splits['valid'].numpy()
test_indices = splits['test'].numpy()

with open('splits.pkl', 'wb') as handle:
    pickle.dump((train_indices, val_indices, test_indices), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('splits.pkl', 'rb') as handle:
    (train_indices, val_indices, test_indices) = pickle.load(handle)

In [ ]:
graph = dataset[0]

## remove some currently unneeded data to free memory

In [ ]:
row, col = graph.edge_index
num_nodes = graph.num_nodes

del dataset, graph

## process adj

In [ ]:
from torch_sparse import SparseTensor
from dataloaders.BaseLoader import BaseLoader

data = torch.ones_like(row, dtype=torch.bool)
adj = SparseTensor(row=row, col=col, value=data, sparse_sizes=(num_nodes, num_nodes))

In [ ]:
adj = adj + adj.t() + SparseTensor.eye(num_nodes, dtype=torch.bool)
adj = BaseLoader.normalize_adjmat(adj, 'sym')

## save adj

In [ ]:
from torch_sparse import SparseTensor

torch.save(adj, '/nfs/students/qian/adj.pt')  # use your /path/to/data

In [ ]:
adj_t = torch.load('/nfs/students/qian/adj.pt')

scipy_adj = adj_t.to_scipy('csr')

## calculate ppr matrices for train, val and test split

See https://github.com/TUM-DAML/pprgo_pytorch/blob/master/pprgo/ppr.py for method reference

In [ ]:
from dataloaders.utils import get_partitions, topk_ppr_matrix

In [ ]:
topk = 96
alpha = 0.05
eps = 2e-5

### val

In [ ]:
edge_index = torch.vstack((adj_t.storage.row(), adj_t.storage.col()))

In [ ]:
num_nodes = adj_t.sizes()[0]

In [ ]:
val_ppr_mat, val_neighbors = topk_ppr_matrix(edge_index,
                    num_nodes,
                    alpha,
                    eps,
                    val_indices,
                    topk)

with open('papers100m_val_ppr.pkl', 'wb') as handle:
    pickle.dump((val_ppr_mat, val_neighbors), handle, protocol=pickle.HIGHEST_PROTOCOL)

### train

In [ ]:
train_ppr_mat, train_neighbors = topk_ppr_matrix(edge_index,
                    num_nodes,
                    alpha,
                    eps,
                    train_indices,
                    topk)

with open('papers100m_train_ppr.pkl', 'wb') as handle:
    pickle.dump((train_ppr_mat, train_neighbors), handle, protocol=pickle.HIGHEST_PROTOCOL)

### test

In [ ]:
test_ppr_mat, test_neighbors = topk_ppr_matrix(edge_index,
                    num_nodes,
                    alpha,
                    eps,
                    test_indices,
                    topk)

with open('papers100m_test_ppr.pkl', 'wb') as handle:
    pickle.dump((test_ppr_mat, test_neighbors), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('papers100m_val_ppr.pkl', 'rb') as handle:
    val_ppr_mat, val_neighbor = pickle.load(handle)

with open('papers100m_train_ppr.pkl', 'rb') as handle:
    train_ppr_mat, train_neighbors = pickle.load(handle)

with open('papers100m_test_ppr.pkl', 'rb') as handle:
    test_ppr_mat, test_neighbors = pickle.load(handle)

## Node-wise IBMB batches

In [ ]:
num_output_node_per_batch = 5000

In [ ]:
from dataloaders.IBMBNodeLoader import get_pairs, prime_orient_merge, prime_post_process

def prime_ppr_loader(ppr_matrix, output_indices, neighbors, num_aux_per_node):
    ppr_matrix = ppr_matrix[:, output_indices]
    ppr_pairs = get_pairs(ppr_matrix)

    output_list = prime_orient_merge(ppr_pairs, num_aux_per_node, len(output_indices))
    output_list = prime_post_process(output_list, num_aux_per_node)
    node_wise_out_aux_pairs = []

    if isinstance(neighbors, list):
        neighbors = np.array(neighbors, dtype=object)

    _union = lambda inputs: np.unique(np.concatenate(inputs))
    for p in output_list:
        node_wise_out_aux_pairs.append((output_indices[p], _union(neighbors[p]).astype(np.int64)))
    return node_wise_out_aux_pairs

In [ ]:
val_loader = prime_ppr_loader(val_ppr_mat, 
                                     val_indices, 
                                     val_neighbor, 
                                     num_output_node_per_batch * 2)

In [ ]:
test_loader = prime_ppr_loader(test_ppr_mat, 
                                     test_indices, 
                                     test_neighbors, 
                                     num_output_node_per_batch * 2)

In [ ]:
train_loader = prime_ppr_loader(train_ppr_mat, 
                                     train_indices, 
                                     train_neighbors, 
                                     num_output_node_per_batch)

In [ ]:
with open('papers100m_train_ppr_batches.pkl', 'wb') as handle:
    pickle.dump(train_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('papers100m_val_ppr_batches.pkl', 'wb') as handle:
    pickle.dump(val_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('papers100m_test_ppr_batches.pkl', 'wb') as handle:
    pickle.dump(test_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Batch-wise IBMB

This is tricky for `ogbn-papers100M` dataset. 

Because the dataset is quite large, METIS partitioning cannot be directly applied. 

For each split, e.g. train split, we obtain some neighborhood of each node, and take the induced subgraph. 

Then we do partitioning on the subgraph.

Finally, we merge the primary nodes in each partition, and auxiliary nodes are obtained from the topk PPR scores. 

In [ ]:
from scipy.sparse import find

In [ ]:
def partition_ppr_loader(partitions, prime_indices, neighbor_list):
    n = len(partitions)
    batches = []
    if isinstance(neighbor_list, list):
        neighbor_list = np.array(neighbor_list, dtype=object)
    for i in range(n):
        intersect = np.intersect1d(partitions[i], prime_indices)
        ind = np.in1d(prime_indices, intersect)
        lst = list(neighbor_list[ind])
        seconds = np.unique(np.concatenate(lst))
        batches.append((intersect, seconds,))
    
    return batches

In [ ]:
thresh = 5e-4
train_parts, val_parts, test_parts = [256, 32, 48]

In [ ]:
for indices, mat, neighbor, num_parts, naming in zip([train_indices, val_indices, test_indices], 
                                             [train_ppr_mat, val_ppr_mat, test_ppr_mat], 
                                             [train_neighbors, val_neighbor, test_neighbors],
                                             [train_parts, val_parts, test_parts],
                                             ['train', 'val', 'test']):
    row, col, val = find(mat)
    
    mask = val > thresh
    mask = np.unique(col[mask])
    torch_mask = torch.from_numpy(mask).long()
    
    temp_adj_t = adj_t[torch_mask, :][:, torch_mask]
    print(f'processed {naming} adj')
    
    _, partptr, perm = temp_adj_t.partition(num_parts=num_parts, recursive=False, weighted=False)
    print(f'partitioned {naming} adj')
    
    partitions = []
    for i in range(len(partptr) - 1):
        partitions.append(mask[perm[partptr[i] : partptr[i + 1]].numpy()])
    print(f'obtained {naming} partitions')
    
    batches = partition_ppr_loader(partitions, indices, neighbor)
    print(f'obtained {naming} batches')
    
    with open(f'papers100m_{naming}_part_batches.pkl', 'wb') as handle:
        pickle.dump(batches, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'saved {naming} batches')

### visualize weight distribution

In [ ]:
row, col, val = find(train_ppr_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(7, 7))
ax.set(xscale="log")
sns.histplot(val, ax=ax, bins=50)